# Build an LLM agent with Claude 3 and Langchain default XML agent configuration

LLM Agents are one of the most impactful design patterns that emerged recently, because it provides a general purpose framework for complex problem solving and task automation. In this notebook, we'll explore how to build an LLM agent that leverages the capabilities of Claude 3 and the default XML agent configuration provided by the LangChain library.

We will also explore the limitations of the default langchain xml agent, and reflect how to improve it.

## Setup

In [1]:
%pip install duckduckgo-search -qU

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -q -U langchain

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -q -U langchainhub langchain-community langchain-aws

Note: you may need to restart the kernel to use updated packages.


In [4]:
import boto3
from langchain_community.chat_models import BedrockChat

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-west-2",
)

bedrock = boto3.client(
    service_name="bedrock",
    region_name="us-west-2",
)

In [5]:
foundation_models = bedrock.list_foundation_models()["modelSummaries"]

In [6]:
claude_models = [model for model in foundation_models if "Claude" in model["modelName"]]

In [7]:
claude_models[-1]

{'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-opus-20240229-v1:0',
 'modelId': 'anthropic.claude-3-opus-20240229-v1:0',
 'modelName': 'Claude 3 Opus',
 'providerName': 'Anthropic',
 'inputModalities': ['TEXT', 'IMAGE'],
 'outputModalities': ['TEXT'],
 'responseStreamingSupported': True,
 'customizationsSupported': [],
 'inferenceTypesSupported': ['ON_DEMAND'],
 'modelLifecycle': {'status': 'ACTIVE'}}

Here's the list of available Claude model ids, we can use the Claude 3 family of models for the agent.

In [8]:
[model["modelId"] for model in claude_models]

['anthropic.claude-instant-v1:2:100k',
 'anthropic.claude-instant-v1',
 'anthropic.claude-v2:0:18k',
 'anthropic.claude-v2:0:100k',
 'anthropic.claude-v2:1:18k',
 'anthropic.claude-v2:1:200k',
 'anthropic.claude-v2:1',
 'anthropic.claude-v2',
 'anthropic.claude-3-sonnet-20240229-v1:0:28k',
 'anthropic.claude-3-sonnet-20240229-v1:0:200k',
 'anthropic.claude-3-sonnet-20240229-v1:0',
 'anthropic.claude-3-haiku-20240307-v1:0:48k',
 'anthropic.claude-3-haiku-20240307-v1:0:200k',
 'anthropic.claude-3-haiku-20240307-v1:0',
 'anthropic.claude-3-opus-20240229-v1:0']

# Initialize Tools

In [9]:
# from langchain import hub
from langchain.agents import AgentExecutor, create_xml_agent
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
tools = [search]

In [10]:
tools

[DuckDuckGoSearchRun()]

The tool object will be rendered into a text representation that will be injected in the agent prompt.
Part of the tool definition is a natural language description that outlines what the tool is, what it can be used for, and what it's input should be.

In [11]:
from langchain.tools.render import ToolsRenderer, render_text_description

In [12]:
render_text_description(tools)

'duckduckgo_search - A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.'

## Define the Agent Prompt

An LLM agent is not formed by making a single call to an LLM, it is rather a system made up of several components that co-operate to give us the overall impression of an intelligent agent. These components are:

1. Agent Planner: Uses the prompt to ask the LLM to plan the next action/tool call that needs to happen to answer the user question and perform the required automation.
2. Agent Executor: Responsible for coordinating the execution of the planned actions, invoking the necessary tools, and handling the overall flow of the agent's decision-making process.
3. Tools: The external capabilities that the agent can leverage to gather information, perform calculations, or take other actions to accomplish the user's request.
4. Output Parser: Responsible for interpreting the LLM responses to identify, extract the tools input if we a tool invocation is required, or the final answer.
5. LLM: The large language model, in this case, the Claude 3 model from Anthropic, which serves as the core intelligence of the agent and provides the reasoning and language generation capabilities.

The aim of the agent desgin pattern is to prompt an LLM to think step by step about a question or task, while interleaving the ability to leverage external tools to perform partial steps or retrieve information.

In [13]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate
)

Below we explore few variants of the agent LLM prompt, gradually being refactoring to enhance how the agent uses the conversation history and to reduce the number of tokens
in the prompt to reduce the cost and inference time.

In [14]:
prompt_template_v1 = """You are a helpful assistant. Help the user answer any questions.

You have access to the following tools:

{tools}

To answer a user question you can use a tool, or leverage the information inside the conversation history within <history> tag.
In order to use a tool, you can use <tool></tool> and <tool_input></tool_input> tags. You will then get back a response in the form <observation></observation>
For example, if you have a tool called 'search' that could run a google search, in order to search for the weather in SF you would respond:

<tool>search</tool><tool_input>weather in SF</tool_input>
<observation>64 degrees</observation>

When you are done, respond with a final answer in markdown between <final_answer></final_answer>.
For example: <final_answer>The weather in SF is 64 degrees</final_answer>

Begin!

Previous Conversation:
<history>
{chat_history}
</history>

Question: {input}
{agent_scratchpad}"""

In [15]:
prompt_template_v2 = """
You are a helpful assistant. Help the user answer any questions, leveraging the information from the conversation history within the <history> tag whenever possible to avoid duplicating work.

You have access to the following tools:

{tools}

To answer a user question, you can either:

1. Reuse relevant information from the conversation history, or
2. Use one or multiple tools

To use a tool, follow this format:

<thinking>Think about what tool to use</thinking>
<tool>tool name</tool><tool_input>valid input to the tool</tool_input>.
<observation>You will then get back a response inside these observation tags</observation>

When you are done, respond with a final answer between <final_answer></final_answer>.

Begin!

Previous Conversation:
<history>
{chat_history}
</history>

Question: {input}
{agent_scratchpad}
"""

In [16]:
prompt_template_v3 = """
You are a helpful assistant. Leverage the <history> to avoid duplicating work when answering questions.

Available tools:
<tools>
{tools}
</tools>

To answer, first review the <history>. If insufficient use tool(s) with the following format:
<thinking>Think about which tool(s) to use and why</thinking>
<tool>tool_name</tool><tool_input>input</tool_input><observation>response</observation>

When you are done, provide a final answer in markdown within <final_answer></final_answer>.

<history>
{chat_history}
</history>

Question: {input}
{agent_scratchpad}
"""

In [17]:
prompt_template = PromptTemplate(
    input_variables=["agent_scratchpad", "chat_history", "input", "tools"],
    template=prompt_template_v3
)

human_message_prompt = HumanMessagePromptTemplate(
    prompt=prompt_template
)

prompt = ChatPromptTemplate(
    input_variables=["agent_scratchpad", "input", "tools"],
    partial_variables={"chat_history": ""},
    messages=[human_message_prompt]
)

In [18]:
from langchain_aws import ChatBedrock

model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
model_id = "anthropic.claude-3-haiku-20240307-v1:0"

model_kwargs =  { 
    "max_tokens": 2048,
    "temperature": 0.0,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

model = ChatBedrock(
    client=bedrock_runtime,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

In [19]:
# Construct the XML Agent
agent = create_xml_agent(
    model, tools, prompt,
    stop_sequence=["</tool_input>", "</final_answer>"]
)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, return_intermediate_steps=True, verbose=True)

In [20]:
%%time
# Invoke XML Agent
response = agent_executor.invoke({"input": "What is Amazon Bedrock?"})



> Entering new AgentExecutor chain...
<thinking>To answer this question, I will first search for information about "Amazon Bedrock" on DuckDuckGo to see if I can find relevant details.</thinking>

<tool>duckduckgo_search</tool><tool_input>Amazon BedrockWith Amazon Bedrock Custom Model Import, organizations can now import and access their own custom models as a fully managed Application Programming Interface (API) in Bedrock, giving them unprecedented choice when building generative AI applications.To get started, organizations can easily add models to Amazon Bedrock that they customized on Amazon SageMaker, or via another third-party tool or ... Update October 10, 2023 — Amazon Bedrock is now available in 3 regions globally: US East (N. Virginia), US West (Oregon), and Asia Pacific (Tokyo). This April, we announced Amazon Bedrock as part of a set of new tools for building with generative AI on AWS.Amazon Bedrock is a fully managed service that offers a choice of high-performing found

In [21]:
print(response['output'])


Amazon Bedrock is a fully managed service from Amazon Web Services (AWS) that allows organizations to build and scale generative AI applications. Key features of Amazon Bedrock include:

- Offers a choice of high-performing foundation models (FMs) from leading AI companies like AI21 Labs, Anthropic, Cohere, Meta, Stability AI, and Amazon.
- Provides a managed API to access and use these foundation models, making it easier to build generative AI applications.
- Supports the latest Llama 2 model from Meta, which has significant improvements over the original Llama models.
- Includes a broad set of capabilities needed to build and scale generative AI applications, such as model management, scaling, and security.

In summary, Amazon Bedrock is a fully managed service that gives organizations the ability to easily build and scale generative AI applications by providing access to a variety of high-quality foundation models through a managed API.



In [22]:
print(response['intermediate_steps'][0][0])
print(response['intermediate_steps'][0][1])

tool='duckduckgo_search' tool_input='Amazon Bedrock' log='<thinking>To answer this question, I will first search for information about "Amazon Bedrock" on DuckDuckGo to see if I can find relevant details.</thinking>\n\n<tool>duckduckgo_search</tool><tool_input>Amazon Bedrock'
With Amazon Bedrock Custom Model Import, organizations can now import and access their own custom models as a fully managed Application Programming Interface (API) in Bedrock, giving them unprecedented choice when building generative AI applications.To get started, organizations can easily add models to Amazon Bedrock that they customized on Amazon SageMaker, or via another third-party tool or ... Update October 10, 2023 — Amazon Bedrock is now available in 3 regions globally: US East (N. Virginia), US West (Oregon), and Asia Pacific (Tokyo). This April, we announced Amazon Bedrock as part of a set of new tools for building with generative AI on AWS.Amazon Bedrock is a fully managed service that offers a choice of

In [23]:
%%time
# Using with chat history
# from langchain_core.messages import AIMessage, HumanMessage

response_with_history = agent_executor.invoke(
    {
        "input": "What is Amazon Bedrock?",
        # Notice that chat_history is a string
        # since this prompt is aimed at LLMs, not chat models
        "chat_history": f"Human: What is Amazon Bedrock?\nAI: {response['intermediate_steps'][0][1]}",
    }
)



> Entering new AgentExecutor chain...
<thinking>The history provided gives a good overview of what Amazon Bedrock is, so I don't need to use any additional tools to answer this question. The key points are:

- Amazon Bedrock is a fully managed service that offers a choice of high-performing foundation models (FMs) from leading AI companies.
- It allows organizations to import and access their own custom models as a fully managed API, giving them more choice when building generative AI applications.
- Amazon Bedrock is now generally available in 3 regions globally and includes support for the latest Llama 2 models from Meta.
- It provides an easy way to build and scale secure generative AI applications using a variety of foundation models.
</thinking>

<final_answer>
Amazon Bedrock is a fully managed service from AWS that allows organizations to build and scale generative AI applications. It offers a choice of high-performing foundation models (FMs) from leading AI companies like AI21

In [24]:
print(response_with_history["output"])


Amazon Bedrock is a fully managed service from AWS that allows organizations to build and scale generative AI applications. It offers a choice of high-performing foundation models (FMs) from leading AI companies like AI21 Labs, Anthropic, Cohere, Meta, Stability AI, and Amazon.

Key features of Amazon Bedrock include:

- Ability to import and access custom models as a fully managed API, giving organizations more choice when building generative AI apps.
- Support for the latest Llama 2 models from Meta, which have significant improvements over the original Llama models.
- Broad set of capabilities to make it easy to build and scale secure generative AI applications.
- General availability in 3 regions globally: US East (N. Virginia), US West (Oregon), and Asia Pacific (Tokyo).

Overall, Amazon Bedrock provides an easy and flexible way for organizations to leverage the power of generative AI in their applications.



In [25]:
%%time
# Using with chat history
# from langchain_core.messages import AIMessage, HumanMessage

response_with_history = agent_executor.invoke(
    {
        "input": "what's Amazon Bedrock Knowledge Base?",
        # Notice that chat_history is a string
        # since this prompt is aimed at LLMs, not chat models
        "chat_history": f"Human: What is Amazon Bedrock?\nAI: {response['intermediate_steps'][0][1]}",
    }
)



> Entering new AgentExecutor chain...
<thinking>
The history provided does not mention anything about an "Amazon Bedrock Knowledge Base". The information given focuses on Amazon Bedrock being a fully managed service that provides access to various foundation models from different AI companies, as well as capabilities for building and scaling generative AI applications. To answer this question, I will need to perform a search to find more information about an "Amazon Bedrock Knowledge Base".
</thinking>

<tool>duckduckgo_search</tool><tool_input>amazon bedrock knowledge baseLearn how to use Knowledge Bases for Amazon Bedrock, a fully managed capability that connects foundation models to internal data sources for Retrieval Augmented Generation (RAG). RAG enhances FM responses with contextual and relevant company data from Amazon S3 or other vector databases. For instructions to create one, see Create a knowledge base. Configure the maximum number of results using the console. To use th

In [26]:
print(response_with_history['output'])
print(response_with_history['intermediate_steps'][0][0])
print(response_with_history['intermediate_steps'][0][1])


Amazon Bedrock Knowledge Base is a fully managed capability within Amazon Bedrock that allows you to connect foundation models (FMs) to your internal data sources for Retrieval Augmented Generation (RAG). RAG enhances the responses from the FMs with contextual and relevant data from your company's data sources, such as Amazon S3 or other vector databases.

The key features of Amazon Bedrock Knowledge Base include:

- Seamless integration of multiple vector stores for data sources
- Ability to ingest data from Amazon S3 and other vector databases
- Fully managed RAG capabilities to enhance FM responses with your company data
- Customization options to fine-tune FMs like Cohere Command, Meta Llama 2, and Amazon Titan models (with Anthropic's Claude coming soon)

With Amazon Bedrock Knowledge Base, you can build generative AI applications that leverage your proprietary data to deliver more relevant, context-specific, and accurate responses, without the need to continuously retrain the fo